# 05 - Function Calling

This notebook demonstrates OpenAI function calling (tool use), where the model can request execution of locally defined functions and incorporate the results into its response.

**Prerequisites:** Set `INSERT_API_KEY_HERE` before running.

In [ ]:
import json
from openai import OpenAI
import os

client = OpenAI(api_key="INSERT_API_KEY_HERE")

## Define Tools & Local Function

A simple calculator tool definition and its Python implementation.

In [ ]:
tools = [
    {
        "type": "function",
        "name": "calculate",
        "description": "Perform a basic arithmetic operation",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "First operand"},
                "b": {"type": "number", "description": "Second operand"},
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "Arithmetic operation to perform"
                }
            },
            "required": ["a", "b", "operation"]
        },
    }
]


def calculate(a: int, b: int, operation: str) -> float:
    if operation == "add":
        return a + b
    if operation == "subtract":
        return a - b
    if operation == "multiply":
        return a * b
    if operation == "divide":
        return a / b
    raise ValueError("Unknown operation")

## Multi-Step Tool Use Loop

The model breaks down `(12.5 * 3) - 7` into multiple function calls. We process each call, feed back the result, and let the model continue until it produces a final answer.

In [ ]:
question = "What is (12.5 * 3) - 7? Only return the result"
input_list = [
    {"role": "user", "content": question}
]

response = client.responses.create(
    model="gpt-5.1",
    input=input_list,
    tools=tools
)

input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "calculate":
            intermediate_result = calculate(**json.loads(item.arguments))

            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                    "intermediate_result": intermediate_result
                })
            })

response = client.responses.create(
    model="gpt-5.1",
    input=input_list,
    tools=tools
)
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "calculate":
            intermediate_result = calculate(**json.loads(item.arguments))

            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                    "intermediate_result": intermediate_result
                })
            })


response = client.responses.create(
    model="gpt-5.1",
    input=input_list,
    tools=tools
)
print(question)
print(input_list)
print(response.output_text)